## Setup

In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive')
print("✓ Google Drive mounted")

Mounted at /content/drive
✓ Google Drive mounted


In [ ]:
import os
os.chdir('/content')

!rm -rf automatic-pain-recognition
!git clone https://github.com/alicka33/automatic-pain-recognition.git

os.chdir('/content/automatic-pain-recognition')
print(f"✓ Repository cloned. Current directory: {os.getcwd()}")

Cloning into 'automatic-pain-recognition'...
remote: Enumerating objects: 333, done.
remote: Counting objects: 100% (56/56), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 333 (delta 28), reused 44 (delta 16), pack-reused 277 (from 1)
Receiving objects: 100% (333/333), 12.41 MiB | 18.71 MiB/s, done.
Resolving deltas: 100% (152/152), done.
✓ Repository cloned. Current directory: /content/automatic-pain-recognition


In [ ]:
!pip install -q -r requirements.txt
!pip install -q pytest
print("✓ Dependencies installed")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 60.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 6.0 MB/s eta 0:00:00
✓ Dependencies installed


In [ ]:
import sys
from pathlib import Path

project_root = Path('/content/automatic-pain-recognition')
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

print(f"✓ Project path updated")

✓ Project path updated


## Run Tests

In [5]:
import tempfile
import shutil
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from pathlib import Path
from torch.utils.data import DataLoader, TensorDataset

from training_utils.train import Trainer
from training_utils.evaluate import Evaluator

In [ ]:
print("\n" + "="*70)
print("TEST 1: Trainer Initialization")
print("="*70)

try:
    class SimpleModel(nn.Module):
        def __init__(self, input_size=50, hidden_size=32, num_classes=3):
            super().__init__()
            self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
            self.fc = nn.Linear(hidden_size, num_classes)

        def forward(self, x):
            _, (h_n, _) = self.lstm(x)
            return self.fc(h_n.squeeze(0))

    # Create dataloaders
    X_train = torch.randn(32, 10, 50)
    y_train = torch.randint(0, 3, (32,))
    train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=8, shuffle=True)

    X_val = torch.randn(16, 10, 50)
    y_val = torch.randint(0, 3, (16,))
    val_loader = DataLoader(TensorDataset(X_val, y_val), batch_size=8, shuffle=False)

    # Create model
    model = SimpleModel(input_size=50, hidden_size=32, num_classes=3)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()

    # Create temp dir for model save
    temp_dir = tempfile.mkdtemp()
    model_path = os.path.join(temp_dir, 'test_model.pt')

    # Create trainer
    trainer = Trainer(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        optimizer=optimizer,
        criterion=criterion,
        device='cpu',
        model_save_path=model_path,
        num_epochs=5,
        monitor='val_acc',
        minimize_monitor=False,
        save_best_only=True
    )

    assert trainer.model is not None
    assert trainer.train_loader is not None
    assert trainer.optimizer is not None
    assert trainer.num_epochs == 5
    print("✓ Trainer initialized successfully")
    print(f"  Monitor: {trainer.monitor}")
    print(f"  Num epochs: {trainer.num_epochs}")
    print(f"  Device: {trainer.device}")

    shutil.rmtree(temp_dir)
    print("\n✓ TEST 1 PASSED")
except Exception as e:
    print(f"✗ TEST 1 FAILED: {e}")
    import traceback
    traceback.print_exc()


TEST 1: Trainer Initialization
✓ Trainer initialized successfully
  Monitor: val_acc
  Num epochs: 5
  Device: cpu

✓ TEST 1 PASSED


In [ ]:
print("\n" + "="*70)
print("TEST 2: Trainer Epoch Methods")
print("="*70)

try:
    # Use existing model and dataloaders from TEST 1

    # Test train_epoch
    train_loss, train_acc = trainer.train_epoch()
    assert isinstance(train_loss, float)
    assert isinstance(train_acc, float)
    assert 0 <= train_acc <= 1
    print(f"✓ train_epoch works")
    print(f"  Train Loss: {train_loss:.4f}")
    print(f"  Train Accuracy: {train_acc:.4f}")

    # Test validate_epoch
    val_loss, val_acc, labels, preds = trainer.validate_epoch()
    assert isinstance(val_loss, float)
    assert isinstance(val_acc, float)
    assert 0 <= val_acc <= 1
    assert len(labels) > 0
    assert len(preds) > 0
    print(f"✓ validate_epoch works")
    print(f"  Val Loss: {val_loss:.4f}")
    print(f"  Val Accuracy: {val_acc:.4f}")

    print("\n✓ TEST 2 PASSED")
except Exception as e:
    print(f"✗ TEST 2 FAILED: {e}")
    import traceback
    traceback.print_exc()


TEST 2: Trainer Epoch Methods
✓ train_epoch works
  Train Loss: 1.1169
  Train Accuracy: 0.3125
✓ validate_epoch works
  Val Loss: 1.1255
  Val Accuracy: 0.1875

✓ TEST 2 PASSED


In [8]:
print("\n" + "="*70)
print("TEST 3: Evaluator")
print("="*70)

try:
    # Create test loader
    X_test = torch.randn(16, 10, 50)
    y_test = torch.randint(0, 3, (16,))
    test_loader = DataLoader(TensorDataset(X_test, y_test), batch_size=8, shuffle=False)

    # Create evaluator
    evaluator = Evaluator(
        model=trainer.model,
        test_loader=test_loader,
        criterion=criterion,
        device='cpu',
        model_name='test_model',
        num_classes=3,
        verbose=False
    )

    # Test evaluate_epoch
    test_loss, test_acc, labels, preds = evaluator.evaluate_epoch()

    assert isinstance(test_loss, float)
    assert isinstance(test_acc, float)
    assert 0 <= test_acc <= 1
    assert len(labels) > 0
    assert len(preds) > 0

    print("✓ Evaluator initialized and evaluated successfully")
    print(f"  Test Loss: {test_loss:.4f}")
    print(f"  Test Accuracy: {test_acc:.4f}")
    print(f"  Num predictions: {len(preds)}")

    print("\n✓ TEST 3 PASSED")
except Exception as e:
    print(f"✗ TEST 3 FAILED: {e}")
    import traceback
    traceback.print_exc()


TEST 3: Evaluator
✓ Evaluator initialized and evaluated successfully
  Test Loss: 1.0648
  Test Accuracy: 0.5000
  Num predictions: 16

✓ TEST 3 PASSED


In [10]:
print("\n" + "="*70)
print("TEST 4: Model Checkpointing")
print("="*70)

try:
    temp_dir = tempfile.mkdtemp()
    checkpoint_dir = os.path.join(temp_dir, 'checkpoints')
    os.makedirs(checkpoint_dir, exist_ok=True)

    # Update trainer's model_save_path to use the checkpoint directory
    model_path = os.path.join(checkpoint_dir, 'checkpoint.pt')
    trainer.model_save_path = model_path

    # Save checkpoint with full path
    saved_path = trainer.save_checkpoint('checkpoint.pt')
    assert os.path.exists(model_path) or os.path.exists(saved_path)
    actual_path = model_path if os.path.exists(model_path) else saved_path
    print(f"✓ Model checkpoint saved")
    print(f"  Path: {actual_path}")

    # Verify file has content
    file_size = os.path.getsize(actual_path)
    assert file_size > 0
    print(f"  Size: {file_size / 1024:.2f} KB")

    # Load checkpoint into a new model
    new_model = SimpleModel(input_size=50, hidden_size=32, num_classes=3)
    state_dict = torch.load(actual_path)
    new_model.load_state_dict(state_dict)
    print(f"✓ Model checkpoint loaded successfully")

    shutil.rmtree(temp_dir)
    print("\n✓ TEST 4 PASSED")
except Exception as e:
    print(f"✗ TEST 4 FAILED: {e}")
    import traceback
    traceback.print_exc()


TEST 4: Model Checkpointing
✓ Model checkpoint saved
  Path: /tmp/tmpv9axo2h3/checkpoints/checkpoint.pt
  Size: 45.37 KB
✓ Model checkpoint loaded successfully

✓ TEST 4 PASSED


## Summary

In [11]:
print("\n" + "="*70)
print("TRAINING UTILS TESTS SUMMARY")
print("="*70)
print("✓ Trainer initialization test passed")
print("✓ Trainer epoch methods test passed")
print("✓ Evaluator test passed")
print("✓ Model checkpointing test passed")
print("\n🎉 All training utility tests completed successfully!")
print("="*70)


TRAINING UTILS TESTS SUMMARY
✓ Trainer initialization test passed
✓ Trainer epoch methods test passed
✓ Evaluator test passed
✓ Model checkpointing test passed

🎉 All training utility tests completed successfully!
